In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets,models
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Consistent input shape handling
#Input
current_input_shape = (227, 227, 3)
new_input_shape = current_input_shape  # No need to modify for this model

In [ ]:
# Model definition
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.layers = nn.Sequential(
            #1
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(96),
            #2
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256),
            #3
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            #4
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            #5
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Flatten
            nn.Flatten(),
            #Flatten 1
            nn.Linear(6 * 6 * 256, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            #Flatten 2
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 7)  # Adjust output units based on dataset
        )
        #Sigmoid
        
    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)

In [ ]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

transform = transforms.Compose([
    transforms.Resize(new_input_shape[:2]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
#Class_name
data_dir = pathlib.Path("/work/Örüntü/Train/")
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)
# classnames in the dataset specified
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt" ])
print(CLASS_NAMES)
# print length of class names
output_class_units = len(CLASS_NAMES)
print(output_class_units)

In [ ]:
# Data loading
data_dir = pathlib.Path("/work/Örüntü/Train/")
data_dir2 = pathlib.Path("/work/Örüntü/Test/")

In [ ]:
#Data_ImageSet
train_dataset = datasets.ImageFolder(data_dir, transform=transform)
test_dataset = datasets.ImageFolder(data_dir2, transform=transform)

In [ ]:
#Data_Loader
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Model Create
model = EmotionClassifier().to(device)
criterion = nn.CrossEntropyLoss()
#Optimizer
optimizer = optim.Adam(model.parameters())

In [ ]:
import time

def print_elapsed_time(start_time, end_time, prefix=""):
    elapsed_time = end_time - start_time
    print(f"{prefix} Elapsed Time: {elapsed_time:.2f} seconds")

# Training loop
num_epochs = 10
print_interval = 100  # Her 10 batch işleminden sonra geçen süreyi yazdır
for epoch in range(num_epochs):
    start_time_epoch = time.time()  # Epoch başlangıç zamanını kaydet

    for i, (images, labels) in enumerate(train_loader):
        batch_start_time = time.time()  # Batch başlangıç zamanını kaydet

        # Eğitim işlemleri burada yapılır

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

        batch_end_time = time.time()  # Batch bitiş zamanını kaydet

        if (i + 1) % print_interval == 0:  # Her print_interval batch işleminden sonra geçen süreyi yazdır
            print_elapsed_time(batch_start_time, batch_end_time, prefix=f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}]")

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')  # or 'macro' for macro F1 score

    print(f"Epoch [{epoch+1}/{num_epochs}] Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    
    end_time_epoch = time.time()  # Epoch bitiş zamanını kaydet
    print_elapsed_time(start_time_epoch, end_time_epoch, prefix=f"Epoch [{epoch+1}/{num_epochs}]")


Epoch [1/10], Batch [100/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [200/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [300/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [400/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [500/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [600/718] Elapsed Time: 0.00 seconds
Epoch [1/10], Batch [700/718] Elapsed Time: 0.00 seconds
Epoch [1/10] Elapsed Time: 3110.62 seconds
Epoch [2/10], Batch [100/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [200/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [300/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [400/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [500/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [600/718] Elapsed Time: 0.00 seconds
Epoch [2/10], Batch [700/718] Elapsed Time: 0.00 seconds
Epoch [2/10] Elapsed Time: 2357.50 seconds
Epoch [3/10], Batch [100/718] Elapsed Time: 0.00 seconds
Epoch [3/10], Batch [200/718] Elapsed Time: 0.00 seconds
Ep

In [ ]:
torch.save(model.state_dict(),'cnn_modelAlexNet.pth')

In [ ]:
model.eval()  # Modeli değerlendirme moduna geçirme
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')  # 'weighted' sınıflar arasındaki dengesizlik için

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

/shared-libs/python3.10/py/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Accuracy: 0.21162579473206175
F1 Score: 0.1373305748490569


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac3974b3-57e3-476c-90ed-f21020147640' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>